In [1]:
from torch.utils.data import DataLoader

from tqdm.notebook import tqdm
import wandb

from WindModel import *

wandb.login()

wandb: Currently logged in as: bhavye-mathur. Use `wandb login --relogin` to force relogin


True

In [3]:
WindDataset.init(0.1)

train = WindDataset("train")
validation = WindDataset("validation")
test = WindDataset("test")

del WindDataset.data

In [4]:
INPUT_SIZE = 15
OUTPUT_SIZE = 1

LOSS_FUNC = torch.nn.MSELoss

In [5]:
def get_dense_model(input_size: int,
                    hidden_sizes: list[int],
                    output_size: int,
                    activation_func: callable):
    layers = []

    for size in hidden_sizes:
        layers.append(torch.nn.Linear(input_size, size))
        layers.append(activation_func())
        input_size = size

    layers.append(torch.nn.Linear(input_size, output_size))

    return torch.nn.Sequential(*layers)

In [6]:
def evaluate_one_epoch(model, epoch):
    mse = 0
    mae = 0

    with torch.no_grad():
        prediction = model(validation.x).squeeze()

        mse += torch.nn.functional.mse_loss(prediction, validation.y)
        mae += torch.nn.functional.l1_loss(prediction, validation.y)

    wandb.log({"val_rmse": (mse ** 0.5) * stds[VARIABLE],
               "val_mae": mae * stds[VARIABLE]})


def train_one_batch(model, optimizer, criterion, batch, batch_idx):
    optimizer.zero_grad()

    inputs, targets = batch

    prediction = model(inputs).squeeze()
    loss = criterion(prediction, targets)

    loss.backward()
    optimizer.step()

    if batch_idx != 0 and batch_idx % 100 == 0:
        with torch.no_grad():
            rmse = (torch.nn.functional.mse_loss(prediction, targets) ** 0.5) * stds[VARIABLE]
            wandb.log({"train_loss": loss,
                       "train_rmse": rmse})


def train_one_epoch(model, optimizer, criterion, epoch, batch_size):
    n = len(train)

    for i in range(len(train) // batch_size):
        lower_i = i * batch_size
        upper_i = min((i + 1) * batch_size, n)

        batch_x = train.x[lower_i: upper_i]
        batch_y = train.y[lower_i: upper_i]

        train_one_batch(model, optimizer, criterion, (batch_x, batch_y), i)


def main():
    wandb.init()

    learning_rate = wandb.config.learning_rate
    batch_size = wandb.config.batch_size
    layers = wandb.config.layers
    epochs = wandb.config.epochs
    activation = wandb.config.activation

    activation = getattr(torch.nn, activation)

    # train_dl = DataLoader(train, batch_size=batch_size, shuffle=True, pin_memory=True)
    # validation_dl = DataLoader(validation, batch_size=batch_size, shuffle=False, pin_memory=True)

    model = get_dense_model(INPUT_SIZE, layers, OUTPUT_SIZE, activation)
    model = model.to(DEVICE)
    print(model)

    criterion = LOSS_FUNC()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    if (scheduler := wandb.config.lr_scheduler) is None:
        scheduler = None
    elif scheduler == "StepLR":
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, **wandb.config.lr_scheduler_kwargs)

    # wandb.watch(model, log_freq=100)

    for ep in tqdm(range(epochs)):
        print(ep, end=" ")

        wandb.log({"epoch": ep})

        model.train()
        train_one_epoch(model, optimizer, criterion, ep, batch_size)

        model.eval()
        evaluate_one_epoch(model, ep)

        if scheduler:
            scheduler.step()
            wandb.log({"lr": scheduler.get_last_lr()[-1]})

In [ ]:
config = {
    "batch_size": 65536,
    "learning_rate": 0.0005,
    "lr_scheduler": "StepLR",
    "lr_scheduler_kwargs": {"step_size": 16, "gamma": 0.3},
    "layers": [512, 256],
    "activation": "PReLU",
    "estimate_quantile": ESTIMATE_QUANTILE,
    "dataset": DATASET,
    "epochs": 50,
}

wandb.init(project=f"MERRA2-{VARIABLE}-July2023", dir="wandb-local", config=config)


main()
wandb.finish()

In [65]:
wandb.finish()

In [ ]:
sweep_configuration = {
    "method": "bayes",
    "name": f"sweep-{DATASET}",
    "metric": {
        "goal": "minimize",
        "name": "val_rmse"
    },
    "parameters": {
        "batch_size": {"values": [8192, 16384, 32768, 65536]},
        "learning_rate": {"max": 0.001, "min": 0.00005},
        "lr_scheduler": {"values": [None, "StepLR"]},
        "lr_scheduler_kwargs": {"parameters": {"step_size": {"max": 20, "min": 10},
                                               "gamma": {"max": 0.75, "min": 0.25}}},
        "layers": {"values": [(256, 128), (512, 256), (1024, 512),
                              (1024, 512, 256), (256, 256, 32), (512, 256, 128), (128, 64, 32)]},
        "epochs": {"value": 40},
        "activation": {"values": ["ReLU", "PReLU", "LeakyReLU", "ELU", "Softplus"]},
        "estimate_quantile": {"value": ESTIMATE_QUANTILE},
        "dataset": {"value": DATASET},
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 3,
        "eta": 2
    }
}

sweep_id = wandb.sweep(sweep=sweep_configuration, project=f"MERRA2-{VARIABLE}-July2023")
wandb.agent(sweep_id, function=main)

Create sweep with ID: 4zc8713b
Sweep URL: https://wandb.ai/bhavye-mathur/MERRA2-U-July2023/sweeps/4zc8713b


wandb: Agent Starting Run: 3qe5o7u9 with config:
wandb: 	activation: LeakyReLU
wandb: 	batch_size: 16384
wandb: 	dataset: NGCT
wandb: 	epochs: 40
wandb: 	estimate_quantile: 0.9935
wandb: 	layers: [1024, 512]
wandb: 	learning_rate: 0.00029286351315302873
wandb: 	lr_scheduler: None
wandb: 	lr_scheduler_kwargs: {'gamma': 0.4672663126655401, 'step_size': 14}


Sequential(
  (0): Linear(in_features=15, out_features=1024, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=1024, out_features=512, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=512, out_features=1, bias=True)
)


  0%|          | 0/40 [00:00<?, ?it/s]

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 

...

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr,█████████████▄▄▄▄▄▄▄▄▄▄▄▄▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▂▃▃▄▄▃▃▃▂▂▂▂▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▂▃▄▄▄▃▃▃▂▂▂▂▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mae,█▄█▇▆▆▅▅▃▂▂▄▃▃▁▂▂▂▂▂▂▂▂▂▂▂▂▃▂▂▁▁▁▁▁▁▁▁▁▁
val_rmse,█▄▆▆▅▅▄▄▃▂▂▄▃▃▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
epoch,39
lr,0.00025
train_loss,0.00168
train_rmse,0.40271
val_mae,0.28434


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: pahhvh3k with config:
wandb: 	activation: ELU
wandb: 	batch_size: 8192
wandb: 	dataset: NGCT
wandb: 	epochs: 40
wandb: 	estimate_quantile: 0.9935
wandb: 	layers: [128, 64, 32]
wandb: 	learning_rate: 0.0004775143714830786
wandb: 	lr_scheduler: StepLR
wandb: 	lr_scheduler_kwargs: {'gamma': 0.4866615041451468, 'step_size': 16}


Sequential(
  (0): Linear(in_features=15, out_features=128, bias=True)
  (1): ELU(alpha=1.0)
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ELU(alpha=1.0)
  (4): Linear(in_features=64, out_features=32, bias=True)
  (5): ELU(alpha=1.0)
  (6): Linear(in_features=32, out_features=1, bias=True)
)


  0%|          | 0/40 [00:00<?, ?it/s]

0 1 

In [ ]:
def test(model, dl):
    model.eval()

    mse = 0
    mae = 0

    with torch.no_grad():
        for inputs, targets in tqdm(dl):
            prediction = model(inputs).squeeze()

            mse += torch.nn.functional.mse_loss(prediction, targets)
            mae += torch.nn.functional.l1_loss(prediction, targets)

    return (mse / len(dl)) ** 0.5 * stds[VARIABLE], (mae / len(dl)) * stds[VARIABLE]


test_dl = DataLoader(test, batch_size=2048, shuffle=False)
test_rmse, test_mae = test(model, test)

print(f"RMSE: {test_rmse} m/s")
print(f"MAE:  {test_mae} m/s")
